In [1]:
#CONEXÃO GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#INSTALANDO BIBLIOTECAS
!pip install pandas

In [3]:
def carregar_dfs(pasta_base, ano,sexo):
  df1 = pd.read_csv(f'{pasta_base}/GERALX{sexo}XADMX{ano}.csv',encoding='latin-1',sep=';',skiprows=3,low_memory=False)     #Mes admissão
  df2 = pd.read_csv(f'{pasta_base}/GERALX{sexo}XDEMX{ano}.csv',encoding='latin-1',sep=';',skiprows=3,low_memory=False)     #Mês demissão
  df3 = pd.read_csv(f'{pasta_base}/GERALX{sexo}XFTEX{ano}.csv',encoding='latin-1',sep=';',skiprows=3,low_memory=False)     #Faixa Tempo de Emprego
  df4 = pd.read_csv(f'{pasta_base}/GERALX{sexo}XATIVOX{ano}.csv',encoding='latin-1',sep=';',skiprows=3,low_memory=False)   #Faixa Ativos
  df5 = pd.read_csv(f'{pasta_base}/GERALX{sexo}XADMXFRMX{ano}.csv',encoding='latin-1',sep=';',skiprows=3,low_memory=False) #Mês adissão x Faixa Remuneração
  df6 = pd.read_csv(f'{pasta_base}/GERALX{sexo}XDEMXFRMX{ano}.csv',encoding='latin-1',sep=';',skiprows=3,low_memory=False) #Mês demissão x Faixa Remuneração
  df7 = pd.read_csv(f'{pasta_base}/GERALX{sexo}XDEMXFTEX{ano}.csv',encoding='latin-1',sep=';',skiprows=3,low_memory=False) #Mês demissão x Tempo de Emprego
  return [df1,df2,df3,df4,df5,df6,df7]

In [4]:
def nome_coluna_uf_cargo(dfs):
    for i in range(len(dfs)):
      if i <= 3:
        dfs[i].rename(columns={'Unnamed: 0':'UF'},inplace=True)
        dfs[i].rename(columns={'Unnamed: 1':'Cargo CBO 2002'},inplace=True)
      else:
        dfs[i].rename(columns={'Unnamed: 0':'Cargo CBO 2002'},inplace=True)
        dfs[i].rename(columns={'Unnamed: 1':'Periodo'},inplace=True)
    return dfs

In [5]:
def remove_linhas(dfs):
    for i in range(len(dfs)):
      if i <= 3:
        dfs[i] = dfs[i].dropna(subset=['Cargo CBO 2002']).reset_index(drop=True)
        dfs[i] = dfs[i][~dfs[i]['Cargo CBO 2002'].str.strip().isin(['Critério','igual a'])].reset_index(drop=True)
      else:
        dfs[i] = dfs[i].dropna(subset=['Periodo']).reset_index(drop=True)
        dfs[i] = dfs[i][~dfs[i]['Periodo'].str.strip().isin(['Critério','igual a'])].reset_index(drop=True)
    return dfs

In [6]:
def completar_UF(dfs):
   for i in range(len(dfs)):
    if i <= 3:
      dfs[i]['UF'] = dfs[i]['UF'].ffill()
      dfs[i]['UF'] = dfs[i]['UF'].replace('{ñ class}', 'N/I')
    else:
      dfs[i]['Cargo CBO 2002'] = dfs[i]['Cargo CBO 2002'].ffill()
   return dfs

In [7]:
def converter_colunas_para_inteiro(dfs):
   colunas_ignoradas = ['UF', 'Cargo CBO 2002','Periodo']
   for i in range(len(dfs)):
        df = dfs[i]
        for col in df.columns:
            if col not in colunas_ignoradas:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
        dfs[i] = df  # garante que a versão modificada seja salva de volta na lista
   return dfs

In [8]:
def Transpor_valor(dfs):
  for i in range(len(dfs)):
    id_vars = list(dfs[i].columns[:2])
    value_vars = [col for col in dfs[i].columns if col not in id_vars]
    if i <= 1:
        var_name = 'Periodo'
    elif i in (2,6) :
        var_name = 'Tempo_Emprego'
    elif i in (3,4,5) :
        var_name = 'Media_Remu'
    df_Temp =dfs[i].melt(
       id_vars=id_vars,
       value_vars=value_vars,
       var_name=var_name,
       value_name='Total')
    df_Temp['Total'] = df_Temp['Total'].astype(int)
    dfs[i] = df_Temp
    df_a = dfs[i]
    df_a.columns
  return dfs

In [9]:
def Linha_A_Linha(dfs):
  for i in range(len(dfs)):
      # Separar as linhas por categorias
      linhas_zero = dfs[i][dfs[i]['Total'] == 0]
      linhas_um = dfs[i][dfs[i]['Total'] == 1]
      linhas_maior_que_um = dfs[i][dfs[i]['Total'] > 1]
      # Repetir as linhas com ValorFaixa > 1 e ajustar
      linhas_repetidas = linhas_maior_que_um.loc[
          linhas_maior_que_um.index.repeat(linhas_maior_que_um['Total'])
      ].copy()
      linhas_repetidas['Total'] = 1
      # concatenando
      df_final = pd.concat([ linhas_um, linhas_repetidas, linhas_zero], ignore_index=True)
      # Ordenando e resetando o índice
      df_final = df_final.sort_values(by=dfs[i].columns.tolist()).reset_index(drop=True)
      dfs[i] = df_final
  return dfs

In [10]:
def Removendo_Totais_0(dfs):
  for i in range(len(dfs)):
    dfs[i] = dfs[i][dfs[i]['Total'] != 0]
  return dfs

In [11]:
def faixas(dfs):
  for i in range(len(dfs)):
    if i <= 1:
        col_name = 'Periodo'
    elif i in (2,6) :
        col_name = 'Tempo_Emprego'
    elif i in (3,4,5) :
        col_name = 'Media_Remu'
    def categorizar_faixa(valor):
            if isinstance(valor, str):
                valor = valor.strip()
                if valor.endswith('.1'):
                    return '15 A 17'
                elif valor.endswith('.2'):
                    return '18 A 24'
                elif valor.endswith('.3'):
                    return '25 A 29'
                elif valor.endswith('.4'):
                    return '30 A 39'
                elif valor.endswith('.5'):
                    return '40 A 49'
                elif valor.endswith('.6'):
                    return '50 A 64'
                elif valor.endswith('.7'):
                    return '65 OU MAIS'
                elif valor.endswith('.8'):
                    return '{ñ class}'
            return '10 A 14'
    dfs[i]['Faixa Etaria'] = dfs[i][col_name].apply(categorizar_faixa)
    dfs[i][col_name] = dfs[i][col_name].str.replace(r'\.\d+$', '', regex=True)
  return dfs

In [12]:
def ajusta_estado(dfs):
  for i in range(len(dfs)):
    coluna = 'UF'
    def limpar_estado(valor):
        if pd.isna(valor):
            return valor
        if str(valor).strip().lower() == '{ñ class}':
            return "N/I"
        return re.sub(r'^\d+\s*-\s*', '', str(valor)).strip()
    if coluna in dfs[i].columns:
      dfs[i][coluna] = dfs[i][coluna].apply(limpar_estado)
  return dfs

In [13]:
def sigla_uf(dfs):
  for i in range(len(dfs)):
    coluna = 'UF'
    estados = {
        'Acre': 'AC','Alagoas': 'AL','Amapá': 'AP','Amazonas': 'AM','Bahia': 'BA','Ceará': 'CE',
    'Espírito Santo': 'ES','Goiás': 'GO','Maranhão': 'MA','Mato Grosso': 'MT','Mato Grosso do Sul': 'MS',
    'Minas Gerais': 'MG','Pará': 'PA','Paraíba': 'PB','Paraná': 'PR','Pernambuco': 'PE','Piauí': 'PI',
    'Rio de Janeiro': 'RJ','Rio Grande do Norte': 'RN','Rio Grande do Sul': 'RS','Rondônia': 'RO','Roraima': 'RR',
    'Santa Catarina': 'SC','São Paulo': 'SP','Sergipe': 'SE','Tocantins': 'TO','Distrito Federal': 'DF','{ñ class}': 'N/I','N/I': 'N/I'
    }
    if coluna in dfs[i].columns:
      dfs[i]['sigla-uf'] = dfs[i]['UF'].map(estados)
  return dfs

In [14]:
def add_ano(dfs,ano):
  for i in range(len(dfs)):
    dfs[i]['Ano'] = ano
  return dfs

In [15]:
def add_sexo(dfs,sexo):
  for i in range(len(dfs)):
    if sexo == 'MAS':
      dfs[i]['Sexo'] = 'Masculino'
    elif sexo == 'FEM':
      dfs[i]['Sexo'] = 'Feminino'
  return dfs

In [16]:
def status(dfs):
    status_map = {0: 'Admitido', 4: 'Admitido',
                  1: 'Demitido', 5: 'Demitido', 6: 'Demitido',
                  2: 'Ativo', 3: 'Ativo'}
    for i, df in enumerate(dfs):
        df['Status'] = status_map.get(i, 'Desconhecido')
    return dfs

In [ ]:
def codigos_cbo(dfs): # ALTERAR DE ACORDO COM AS FAMILIAS ESCOLHIDAS E NOMES DE ARQUIVOS SALVOS
  for i in range(len(dfs)):
    cargos = {'ENGENHEIRO DE APLICATIVOS EM COMPUTACAO':'2122-05',
              'ENGENHEIRO DE EQUIPAMENTOS EM COMPUTACAO':'2122-10',
              'ENGENHEIROS DE SISTEMAS OPERACIONAIS EM COMPUTACAO':'2122-15',
              'ADMINISTRADOR DE BANCO DE DADOS':'2123-05' ,
              'ADMINISTRADOR DE REDES':'2123-10' ,
              'ADMINISTRADOR DE SISTEMAS OPERACIONAIS':'2123-15',
              'ADMINISTRADOR EM SEGURANÇA DA INFORMAÇÃO':'2123-20',
              'ANALISTA DE DESENVOLVIMENTO DE SISTEMAS':'2124-05',
              'ANALISTA DE REDES E DE COMUNICACAO DE DADOS':'2124-10',
              'ANALISTA DE SISTEMAS DE AUTOMACAO':'2124-15',
              'ANALISTA DE SUPORTE COMPUTACIONAL':'2124-20',
              'ARQUITETO DE SOLUÇÕES DE TECNOLOGIA DA INFORMAÇÃO':'2124-25',
              'ANALISTA DE TESTES DE TECNOLOGIA DA INFORMAÇÃO':'2124-30',
              'DIRETOR DE SERVICOS DE INFORMATICA':'1236-05',
              'PROGRAMADOR DE INTERNET':'3171-05',
              'PROGRAMADOR DE SISTEMAS DE INFORMACAO':'3171-10',
              'PROGRAMADOR DE MAQUINAS - FERRAMENTA COM COMANDO NUMERICO':'3171-15',
              'PROGRAMADOR DE MULTIMIDIA':'3171-20',
              'OPERADOR DE COMPUTADOR (INCLUSIVE MICROCOMPUTADOR)':'3172-05',
              'TECNICO DE APOIO AO USUARIO DE INFORMATICA (HELPDESK)':'3172-10',
              'GERENTE DE REDE':'1425-05',
              'GERENTE DE DESENVOLVIMENTO DE SISTEMAS':'1425-10',
              'GERENTE DE PRODUCAO DE TECNOLOGIA DA INFORMACAO':'1425-15',
              'GERENTE DE PROJETOS DE TECNOLOGIA DA INFORMACAO':'1425-20',
              'GERENTE DE SEGURANCA DE TECNOLOGIA DA INFORMACAO':'1425-25',
              'GERENTE DE SUPORTE TECNICO DE TECNOLOGIA DA INFORMACAO':'1425-30',
              'TECNÓLOGO EM GESTÃO DA TECNOLOGIA DA INFORMAÇÃO':'1425-35'}
    dfs[i]['Cod CBO 2002'] = dfs[i]['Cargo CBO 2002'].map(cargos)
  return dfs

In [ ]:
def familia_cbo(dfs): # ALTERAR DE ACORDO COM AS FAMILIAS ESCOLHIDAS E NOMES DE ARQUIVOS SALVOS
  for i in range(len(dfs)):
    familia = {
        '1236': 'DIRETORES DE SERVICOS DE INFORMATICA',
        '2122': 'ENGENHEIROS EM COMPUTACAO',
        '2123': 'ESPECIALISTAS EM INFORMATICA',
        '2124': 'ANALISTA DE SISTEMAS COMPUTACIONAIS',
        '3171': 'TECNICOS EM PROGRAMACAO',
        '3172': 'TECNICOS EM OPERACAO E MONITORAMENTO DE COMPUTADORES',
        '1425': 'GERENTES DE TECNOLOGIA DA INFORMACAO'}
    dfs[i]['Familia CBO 2002'] = dfs[i]['Cod CBO 2002'].astype(str).str[:4].map(familia)
  return dfs

In [19]:
def ordena_ativos(dfs):
  for i in range(len(dfs)):
    dfs[i] = dfs[i].sort_values(by=['UF', 'Cargo CBO 2002','Faixa Etaria'], ascending=[True, True, True]).reset_index(drop=True)
  return dfs

In [20]:
def concatena_ativos(dfs,ano):
  dfs_temp = [dfs[2],dfs[3]]
  dfs_temp = ordena_ativos(dfs_temp)
  df_ativos = pd.concat([dfs_temp[0][['Cargo CBO 2002','Faixa Etaria','UF','Tempo_Emprego','Total','sigla-uf',
  'Ano','Status','Sexo','Cod CBO 2002','Familia CBO 2002']].reset_index(drop=True),dfs_temp[1][['Media_Remu']].reset_index(drop=True)],axis=1)
  df_ativos['Periodo']= ano
  df_ativos = df_ativos[['UF', 'sigla-uf', 'Ano', 'Periodo', 'Faixa Etaria', 'Sexo',
  'Familia CBO 2002', 'Cargo CBO 2002', 'Cod CBO 2002', 'Status', 'Tempo_Emprego', 'Media_Remu', 'Total']]
  return df_ativos

In [21]:
def ordena_adm_dem(dfs):
  for i in range(len(dfs)):
    dfs[i] = dfs[i].sort_values(by=['Cargo CBO 2002','Faixa Etaria','Periodo'], ascending=[True, True, True]).reset_index(drop=True)
  return dfs

In [22]:
def concatena_adm(dfs):
  dfs_temp = [dfs[0],dfs[4]]
  for i in range(len(dfs_temp)):
    dfs_temp[i]['Periodo'] = dfs_temp[i]['Periodo'].replace('{ñ class}', 'N/I').fillna('Não admitido ano')
  dfs_temp = ordena_adm_dem(dfs_temp)
  df_adm = pd.concat([dfs_temp[0][['Cargo CBO 2002','Faixa Etaria','UF','Total','sigla-uf','Ano','Status',
  'Sexo','Cod CBO 2002','Familia CBO 2002','Periodo']].reset_index(drop=True),dfs_temp[1][['Media_Remu']].reset_index(drop=True)],axis=1)
  df_adm['Tempo_Emprego']= '-'
  df_adm = df_adm[['UF', 'sigla-uf', 'Ano', 'Periodo', 'Faixa Etaria', 'Sexo', 'Familia CBO 2002', 'Cargo CBO 2002',
                   'Cod CBO 2002', 'Status', 'Tempo_Emprego', 'Media_Remu', 'Total']]
  return df_adm


In [23]:
def concatena_dem(dfs):
  dfs_temp = [dfs[1],dfs[5],dfs[6]]
  for i in range(len(dfs_temp)):
    dfs_temp[i]['Periodo'] = dfs_temp[i]['Periodo'].replace('{ñ class}', 'N/I').fillna('Não desligado ano')
  dfs_temp = ordena_adm_dem(dfs_temp)
  df_dem = pd.concat([dfs_temp[0][['Cargo CBO 2002','Faixa Etaria','UF','Total','sigla-uf','Ano','Status',
  'Sexo','Cod CBO 2002','Familia CBO 2002','Periodo']].reset_index(drop=True),dfs_temp[1][['Media_Remu']].reset_index(drop=True)],axis=1)
  df_dem = pd.concat([df_dem[['Cargo CBO 2002','Faixa Etaria','UF','Total','sigla-uf','Ano','Status',
  'Sexo','Cod CBO 2002','Familia CBO 2002','Periodo','Media_Remu']].reset_index(drop=True),dfs_temp[2][['Tempo_Emprego']].reset_index(drop=True)],axis=1)
  df_dem = df_dem[['UF', 'sigla-uf', 'Ano', 'Periodo', 'Faixa Etaria', 'Sexo', 'Familia CBO 2002', 'Cargo CBO 2002',
  'Cod CBO 2002', 'Status', 'Tempo_Emprego', 'Media_Remu', 'Total']]
  return df_dem


In [24]:
def concatenando_dfs(dfs,ano):
  df_ativos = concatena_ativos(dfs,ano)
  df_adm = concatena_adm(dfs)
  df_dem = concatena_dem(dfs)
  df_final = pd.concat([df_adm,df_dem,df_ativos],axis=0)
  return df_final

In [ ]:
import pandas as pd
import numpy as np
import re
ano = 2015
pasta_raiz = '/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Cargos_CBO_Dados' #ALTERAR PASTA
pasta_anos = '/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Cargos_CBO_Manipulados' #ALTERAR PASTA
while ano <= 2024:
  a = 0
  pasta_base = f'{pasta_raiz}/{ano}'
  while a <= 1:
        if a == 0:
          sexo = 'MAS'
        elif a == 1:
          sexo = 'FEM'
        print(ano,sexo)
        dfs=[]
        dfs = carregar_dfs(pasta_base, ano,sexo)
        df1,df2,df3,df4,df5,df6,df7 = dfs
        dfs = [df1,df2,df3,df4,df5,df6,df7]
        dfs = nome_coluna_uf_cargo(dfs)
        dfs = remove_linhas(dfs)
        dfs = completar_UF(dfs)
        dfs = converter_colunas_para_inteiro(dfs)
        dfs = Transpor_valor(dfs)
        dfs = Linha_A_Linha(dfs)
        dfs = Removendo_Totais_0(dfs)
        dfs = faixas(dfs)
        dfs = ajusta_estado(dfs)
        dfs = sigla_uf(dfs)
        dfs = add_ano(dfs,ano)
        dfs = status(dfs)
        dfs = add_sexo(dfs,sexo)
        dfs = codigos_cbo(dfs)
        dfs = familia_cbo(dfs)
        df1,df2,df3,df4,df5,df6,df7 = dfs
        df = concatenando_dfs(dfs,ano)
        df.to_csv(f'{pasta_anos}/{sexo}X{ano}.csv',index=False,sep=';', encoding='utf-8')
        print('salvo')
        del df1, df2, df3, df4,df5,df6,df7,df,dfs
        a += 1
  ano += 1

2015 MAS
salvo
2015 FEM
salvo
2016 MAS
salvo
2016 FEM
salvo
2017 MAS
salvo
2017 FEM
salvo
2018 MAS
salvo
2018 FEM
salvo
2019 MAS
salvo
2019 FEM
salvo
2020 MAS
salvo
2020 FEM
salvo
2021 MAS
salvo
2021 FEM
salvo
2022 MAS
salvo
2022 FEM
salvo
2023 MAS
salvo
2023 FEM
salvo
2024 MAS
salvo
2024 FEM
salvo
